In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import expr
from pyspark.ml.stat import Summarizer
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Flight_Delay_Jan_2020_ontime.csv")

In [ ]:
spark = (SparkSession
       .builder
       .appName("FlightDataStatistics")
       .getOrCreate())

In [ ]:
pysparkDF = spark.read.option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", ",") \
        .csv(inputFile) \
        .withColumn("DIVERTED_BOOL", expr("DIVERTED").cast(BooleanType())) \
        .withColumn("CANCELLED_BOOL", expr("CANCELLED").cast(BooleanType())) \
        .withColumn("DEP_DEL15_BOOL", expr("DEP_DEL15").cast(BooleanType())) \
        .withColumn("ARR_DEL15_BOOL", expr("ARR_DEL15").cast(BooleanType())) \
        
pysparkDF.printSchema()

In [ ]:
# DATA UNDERSTANDING!
# Check whether OP_CARRIER_FL_NUM is merely running id for flights or rather encoding specific trims (e.g. Istanbul -> New York)
pysparkDF.groupby('OP_CARRIER_FL_NUM').count().show()

In [ ]:
# DATA UNDERSTANDING!
# A tail number refers to an identification number painted on an aircraft, frequently on the tail.
# Check amount of flights per plane
pysparkDF.groupby('TAIL_NUM').count().show()

# Check average flights per plane per year
pysparkDF.groupby('TAIL_NUM').count().agg(F.mean('count')).show()

In [ ]:
# DATA UNDERSTANDING!
# Check whether ORIGIN_AIRPORT_ID is 1:1 mapping onto ORIGIN
pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN']).count().count() == pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN']).count().dropDuplicates(['ORIGIN_AIRPORT_ID']).count()

# -> ORIGIN_AIRPORT_ID is string indexing ORIGIN

In [ ]:
# DATA UNDERSTANDING!
# Check whether ORIGIN_AIRPORT_ID is 1:1 mapping onto ORIGIN_AIRPORT_SEQ_ID
pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID']).count().count() == pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID']).count().dropDuplicates(['ORIGIN_AIRPORT_ID']).count()

# -> ORIGIN_AIRPORT_ID is 1:1 mapping to ORIGIN_AIRPORT_SEQ_ID

In [ ]:
# DATA UNDERSTANDING!
# Check whether DEST_AIRPORT_ID is 1:1 mapping onto DEST
pysparkDF.groupby(['DEST_AIRPORT_ID','DEST']).count().count() == pysparkDF.groupby(['DEST_AIRPORT_ID','DEST']).count().dropDuplicates(['DEST_AIRPORT_ID']).count()

# -> DEST_AIRPORT_ID is string indexing DEST

In [ ]:
# DATA UNDERSTANDING!
# Check whether DEST_AIRPORT_ID is 1:1 mapping onto DEST_AIRPORT_SEQ_ID
pysparkDF.groupby(['DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID']).count().count() == pysparkDF.groupby(['DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID']).count().dropDuplicates(['DEST_AIRPORT_SEQ_ID']).count()

# -> DEST_AIRPORT_ID is 1:1 mapping to ORIGIN_AIRPORT_SEQ_ID

In [ ]:
# DATA UNDERSTANDING!
# Check whether OP_UNIQUE_CARRIER is 1:1 mapping onto OP_CARRIER
pysparkDF.groupby(['OP_UNIQUE_CARRIER','OP_CARRIER']).count().count() == pysparkDF.groupby(['OP_UNIQUE_CARRIER','OP_CARRIER']).count().dropDuplicates(['OP_UNIQUE_CARRIER']).count()

# -> OP_UNIQUE_CARRIER is 1:1 mapping to OP_CARRIER

In [ ]:
# DATA UNDERSTANDING!
# Check whether TAIL_NUM is 1:1 mapping onto OP_CARRIER_FL_NUM
pysparkDF.groupby(['TAIL_NUM','OP_CARRIER_FL_NUM']).count().count() == pysparkDF.groupby(['TAIL_NUM','OP_CARRIER_FL_NUM']).count().dropDuplicates(['OP_CARRIER_FL_NUM']).count()

# -> TAIL_NUM is not 1:1 mapping to OP_CARRIER_FL_NUM -> One distinct plane can fly multiple routes

### Remove faulty features

In [ ]:
pysparkDF = pysparkDF.drop('_c21')
pysparkDF.printSchema()

### Remove records containing NULLvalues

In [ ]:
pysparkDF_nonull = pysparkDF.dropna()
f"Removed {pysparkDF.count()-pysparkDF_nonull.count()} records containing NULL values"

### Build String indexer for TAIL_NUM

In [ ]:
tailNum_Indexer = StringIndexer().setInputCol("TAIL_NUM").setOutputCol("TAIL_NUM_ID").fit(pysparkDF_nonull)
pysparkDF_indexed = tailNum_Indexer.transform(pysparkDF_nonull)

### Define label columns and remove non-BOOL label columns

In [ ]:
labelCols = ["DEP_DEL15_BOOL","ARR_DEL15_BOOL","CANCELLED_BOOL","DIVERTED_BOOL"]

pysparkDF_nonBOOL = pysparkDF_indexed
for col in [label[:-5] for label in labelCols]:
    pysparkDF_nonBOOL = pysparkDF_nonBOOL.drop(col)
pysparkDF_nonBOOL.printSchema()

### Remove redundant features and labels for unconditional prediction
-> Unconditional is referring to predicting each of the labels without having information on the current status of the flight (Use-Case: Checking the day before)

In [ ]:
# Remove strings from id/string pairs (redundant)
# Remark: since in this dataset both string and id exist already, no further preprocessing via string_indexer is necessary. Otherwise, strings would have first been converted to ids via string_indexer.
featureCols_unconditional = pysparkDF_nonBOOL.columns.copy()
featureCols_unconditional.remove("TAIL_NUM") # -> TAIL_NUM_ID
featureCols_unconditional.remove("OP_UNIQUE_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_unconditional.remove("OP_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_unconditional.remove("ORIGIN") # -> ORIGIN_AIRPORT_ID
featureCols_unconditional.remove("ORIGIN_AIRPORT_SEQ_ID") # -> ORIGIN_AIRPORT_ID
featureCols_unconditional.remove("DEST") # -> DEST_AIRPORT_SEQ_ID
featureCols_unconditional.remove("DEST_AIRPORT_SEQ_ID") # -> DEST_AIRPORT_SEQ_ID
featureCols_unconditional.remove("DEP_TIME_BLK") # -> preliminary elimination, check if model works better with binned values or not

for label in labelCols:
    featureCols_unconditional.remove(label)
                                     
featureCols_unconditional                         

### Remove redundant features and labels for conditional prediction
-> Conditional is referring to predicting each of the labels considering available real-time information on the current status of the flight (Use-Case: Checking while at the airport, pre-flight)

One would expect that prediction performance is increased when the model is aware of the current flight status (=DEP_DEL15)

Example: If the model is aware that the flight has departure delay, it might be able to better predict whether it will also be delayed at arrival

In [ ]:
# Remove strings from id/string pairs (redundant)
# Remark: since in this dataset both string and id exist already, no further preprocessing via string_indexer is necessary. Otherwise, strings would have first been converted to ids via string_indexer.
featureCols_conditional = pysparkDF_nonBOOL.columns.copy()
featureCols_conditional.remove("TAIL_NUM") # -> TAIL_NUM_ID
featureCols_conditional.remove("OP_UNIQUE_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_conditional.remove("OP_CARRIER") # -> OP_CARRIER_AIRLINE_ID
featureCols_conditional.remove("ORIGIN") # -> ORIGIN_AIRPORT_ID
featureCols_conditional.remove("ORIGIN_AIRPORT_SEQ_ID") # -> ORIGIN_AIRPORT_ID
featureCols_conditional.remove("DEST") # -> DEST_AIRPORT_SEQ_ID
featureCols_conditional.remove("DEST_AIRPORT_SEQ_ID") # -> DEST_AIRPORT_SEQ_ID
featureCols_conditional.remove("DEP_TIME_BLK") # -> preliminary elimination, check if model works better with binned values or not

for label in [label for label in labelCols if label!="DEP_DEL15"]:
    featureCols_conditional.remove(label)
    
featureCols_conditional

### Build and apply feature column assembler for both featureCols

In [ ]:
assembler_unconditional =  VectorAssembler(outputCol="features", inputCols=list(featureCols_unconditional))
assembler_conditional =  VectorAssembler(outputCol="features", inputCols=list(featureCols_conditional))

featureSet_unconditional = assembler_unconditional.transform(pysparkDF_nonBOOL)
featureSet_conditional = assembler_conditional.transform(pysparkDF_nonBOOL)

# Define same base-scaler for both feature cols
scaler = StandardScaler(inputCol="features",
                        outputCol="scaledFeatures",
                        withStd=True, 
                        withMean=False)

# Compute summary statistics by fitting the StandardScaler
scalerModel_unconditional = scaler.fit(featureSet_unconditional)
scalerModel_conditional = scaler.fit(featureSet_conditional)

scaledFeatureSet_unconditional = scalerModel_unconditional.transform(featureSet_unconditional)
scaledFeatureSet_conditional = scalerModel_conditional.transform(featureSet_conditional)

### Split data into training and test set

In [ ]:
splits_unconditional = scaledFeatureSet_unconditional.randomSplit([0.9, 0.1 ], 12345)
training_unconditional = splits_unconditional[0]
test_unconditional = splits_unconditional[1]

splits_conditional= scaledFeatureSet_conditional.randomSplit([0.9, 0.1 ], 12345)
training_conditional = splits_conditional[0]
test_conditional = splits_conditional[1]

In [ ]:
spark.stop()